## A simple game

We need to define actions, states, state transitions, rewards and the discount factor.

An MDP is a 5-tuple, $\langle S, A, R, P, \gamma \rangle$

--16 states

--4 actions



In [1]:
import numpy as np

# Define the gridworld environment
n_states = 16
n_actions = 4
P = np.zeros((n_states, n_actions, n_states))  # transition probabilities
R = np.zeros((n_states, n_actions, n_states))  # rewards
gamma = 0.9  # discount factor

# Fill in the transition probabilities and rewards
for s in range(n_states):
    for a in range(n_actions):
        if s == 0 or s == 15:
            P[s, a, s] = 1
        else:
            if a == 0:  # up
                s_prime = s - 4
            elif a == 1:  # down
                s_prime = s + 4
            elif a == 2:  # left
                s_prime = s - 1
            else:  # right
                s_prime = s + 1
            if s_prime < 0:
              s_prime = 0
            if s_prime > 15:
              s_prime = 15

            if s_prime == 0:
                R[s, a, s_prime] = -1  # start state
            elif s_prime == 15:
                R[s, a, s_prime] = 10  # goal state
            else:
                R[s, a, s_prime] = -1  # other states
            P[s, a, s_prime] = 1

print(P)
# P[s, a, s_prime]


[[[1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [1. 0. 0. ... 0. 0. 0.]
  [0. 0. 1. ... 0. 0. 0.]]

 [[1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 1. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 1. 0. 0.]
  [0. 0. 0. ... 0. 0. 1.]]

 [[0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]
  [0. 0. 0. ... 0. 0. 1.]]]


## Solution - DP

In [ ]:
# Define the policy (arbitrary for now)
policy = np.ones((n_states, n_actions)) / n_actions

print(policy)
for s in range(n_states):
  print("(State ", s, ") Actions", policy[s])

# Policy evaluation algorithm
V = np.zeros(n_states)  # initial value function estimate
print("Value function:")
print(V.reshape(4, 4))
tolerance = 1e-6  # convergence tolerance
while True:
    delta = 0
    for s in range(n_states):
        v = V[s]
        bellman_update = 0
        for a in range(n_actions):
            for s_prime in range(n_states):
                bellman_update += policy[s, a] * P[s, a, s_prime] * (R[s, a, s_prime] + gamma * V[s_prime])
        V[s] = bellman_update
        delta = max(delta, abs(v - V[s]))
    if delta < tolerance:
        break

print("Value function:")
print(V.reshape(4, 4))

## Q-learning algorithm

In [5]:
Q = np.zeros((n_states, n_actions))  # initial Q-values
n_episodes = 10000  # number of episodes
alpha = 0.1  # learning rate
epsilon = 0.1  # epsilon-greedy exploration probability

for episode in range(n_episodes):
    s = np.random.randint(n_states)  # randomly select initial state
    while s not in [0, 15]:
        # epsilon-greedy action selection
        if np.random.uniform() < epsilon:
            a = np.random.randint(n_actions)
        else:
            a = np.argmax(Q[s, :])
        # take the selected action and observe the next state and reward
        s_prime = np.random.choice(range(n_states), p=P[s, a, :])
        r = R[s, a, s_prime]
        # update Q-value for the current state-action pair
        Q[s, a] = Q[s, a] + alpha * (r + gamma * np.max(Q[s_prime, :]) - Q[s, a])
        s = s_prime

# Extract the optimal policy from Q-values
policy = np.argmax(Q, axis=1)

print("Optimal policy:")
print(policy.reshape(4, 4))


Optimal policy:
[[0 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 0]]
